In [43]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import json
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from math import sqrt

In [5]:
# read data
training_x = list()
with open ("../training_X_with_text_features.json",'r') as data:
    for d in data:
        training_x = json.loads(d)
        
# read data
training_y = list()
with open ("../training_Y_with_text_features.json",'r') as data:
    for d in data:
        training_y = json.loads(d)
        
training_x = np.array(training_x)
training_y = np.array(training_y)

print(training_x.shape)
print(training_y.shape)

# read data
testing_x = list()
with open ("../testing_X_with_text_features.json",'r') as data:
    for d in data:
        testing_x = json.loads(d)
        
# read data
testing_y = list()
with open ("../testing_Y_with_text_features.json",'r') as data:
    for d in data:
        testing_y = json.loads(d)
        
testing_x = np.array(testing_x)
testing_y = np.array(testing_y)

print(testing_x.shape)
print(testing_y.shape)

(1237673, 20)
(1237673,)
(280588, 20)
(280588,)


In [32]:
# use 30% of training data to do parameter tuning
part_training_x = training_x[:round(0.3*1237673),:]
part_training_y = training_y[:round(0.3*1237673)]
print(part_training_x.shape)
print(part_training_y.shape)

(371302, 20)
(371302,)


In [33]:
max_depth_list = [6,8,10,12,16,20]
n_estimators_list =  [50,100,200,300,400,500]
learning_rate_list =  [0.01, 0.05]
num_leaves_list =  [20,25,30,35,40]

In [34]:
results_dict = dict()
for a in max_depth_list:
    for b in n_estimators_list:
        for c in learning_rate_list:
            for d in num_leaves_list:
                
                gbm = lgb.LGBMRegressor(objective="regression", boosting_type='gbdt', random_state=2020, 
                                        n_jobs=-1, max_depth=a, n_estimators=b, learning_rate=c,
                                        num_leaves= d, reg_alpha = 0.5, reg_lambda=0.5)

                cv_results = cross_validate(gbm, part_training_x, part_training_y, cv=3, scoring=('neg_mean_squared_error'))
                results = -cv_results['test_score']
                key = "{},{},{},{}".format(a,b,c,d)
                results_dict[key] = round(np.mean(results),5)
                print(round(np.mean(results),3), "{},{},{},{}".format(a,b,c,d))

1.218 6,50,0.01,20
1.217 6,50,0.01,25
1.217 6,50,0.01,30
1.216 6,50,0.01,35
1.216 6,50,0.01,40
1.142 6,50,0.05,20
1.141 6,50,0.05,25
1.14 6,50,0.05,30
1.14 6,50,0.05,35
1.139 6,50,0.05,40
1.173 6,100,0.01,20
1.172 6,100,0.01,25
1.171 6,100,0.01,30
1.17 6,100,0.01,35
1.17 6,100,0.01,40
1.136 6,100,0.05,20
1.135 6,100,0.05,25
1.135 6,100,0.05,30
1.134 6,100,0.05,35
1.134 6,100,0.05,40
1.146 6,200,0.01,20
1.145 6,200,0.01,25
1.144 6,200,0.01,30
1.144 6,200,0.01,35
1.143 6,200,0.01,40
1.134 6,200,0.05,20
1.133 6,200,0.05,25
1.133 6,200,0.05,30
1.133 6,200,0.05,35
1.132 6,200,0.05,40
1.14 6,300,0.01,20
1.139 6,300,0.01,25
1.138 6,300,0.01,30
1.138 6,300,0.01,35
1.137 6,300,0.01,40
1.133 6,300,0.05,20
1.133 6,300,0.05,25
1.133 6,300,0.05,30
1.133 6,300,0.05,35
1.133 6,300,0.05,40
1.137 6,400,0.01,20
1.136 6,400,0.01,25
1.136 6,400,0.01,30
1.135 6,400,0.01,35
1.135 6,400,0.01,40
1.133 6,400,0.05,20
1.133 6,400,0.05,25
1.133 6,400,0.05,30
1.133 6,400,0.05,35
1.133 6,400,0.05,40
1.136 6,500,0.0

In [37]:
sorted(results_dict.items(), key=lambda x:x[1], reverse=False)[:10]

[('10,300,0.05,35', 1.13239),
 ('6,200,0.05,40', 1.13248),
 ('12,300,0.05,35', 1.13252),
 ('16,200,0.05,40', 1.13253),
 ('16,300,0.05,35', 1.13254),
 ('12,200,0.05,35', 1.13256),
 ('10,400,0.05,35', 1.13257),
 ('10,200,0.05,35', 1.13258),
 ('12,200,0.05,40', 1.13258),
 ('20,200,0.05,40', 1.13259)]

In [ ]:
# we continue tuning the model, with fixing n_estimators_list to 200/300, learning rate=0.05
# num_leves seems the larger the better
# max depth seems reasonable

In [38]:
max_depth_list = [6,8,10,12,16,20]
n_estimators_list =  [200, 250,300]
learning_rate_list =  [0.05]
num_leaves_list =  [30,35,40,45,50]

In [39]:
results_dict = dict()
for a in max_depth_list:
    for b in n_estimators_list:
        for c in learning_rate_list:
            for d in num_leaves_list:
                
                gbm = lgb.LGBMRegressor(objective="regression", boosting_type='gbdt', random_state=2020, 
                                        n_jobs=-1, max_depth=a, n_estimators=b, learning_rate=c,
                                        num_leaves= d, reg_alpha = 0.5, reg_lambda=0.5)

                cv_results = cross_validate(gbm, part_training_x, part_training_y, cv=3, scoring=('neg_mean_squared_error'))
                results = -cv_results['test_score']
                key = "{},{},{},{}".format(a,b,c,d)
                results_dict[key] = round(np.mean(results),5)
                print(round(np.mean(results),3), "{},{},{},{}".format(a,b,c,d))

1.133 6,200,0.05,30
1.133 6,200,0.05,35
1.132 6,200,0.05,40
1.133 6,200,0.05,45
1.133 6,200,0.05,50
1.133 6,250,0.05,30
1.133 6,250,0.05,35
1.132 6,250,0.05,40
1.133 6,250,0.05,45
1.133 6,250,0.05,50
1.133 6,300,0.05,30
1.133 6,300,0.05,35
1.133 6,300,0.05,40
1.133 6,300,0.05,45
1.133 6,300,0.05,50
1.133 8,200,0.05,30
1.133 8,200,0.05,35
1.133 8,200,0.05,40
1.133 8,200,0.05,45
1.132 8,200,0.05,50
1.133 8,250,0.05,30
1.133 8,250,0.05,35
1.133 8,250,0.05,40
1.133 8,250,0.05,45
1.132 8,250,0.05,50
1.133 8,300,0.05,30
1.133 8,300,0.05,35
1.133 8,300,0.05,40
1.133 8,300,0.05,45
1.133 8,300,0.05,50
1.133 10,200,0.05,30
1.133 10,200,0.05,35
1.133 10,200,0.05,40
1.133 10,200,0.05,45
1.133 10,200,0.05,50
1.133 10,250,0.05,30
1.132 10,250,0.05,35
1.133 10,250,0.05,40
1.133 10,250,0.05,45
1.133 10,250,0.05,50
1.133 10,300,0.05,30
1.132 10,300,0.05,35
1.133 10,300,0.05,40
1.133 10,300,0.05,45
1.133 10,300,0.05,50
1.133 12,200,0.05,30
1.133 12,200,0.05,35
1.133 12,200,0.05,40
1.133 12,200,0.05,45
1

In [41]:
sorted(results_dict.items(), key=lambda x:x[1], reverse=False)[:5]

[('10,300,0.05,35', 1.13239),
 ('10,250,0.05,35', 1.13241),
 ('8,200,0.05,50', 1.13242),
 ('6,250,0.05,40', 1.13244),
 ('12,250,0.05,35', 1.13245)]

In [42]:
# train the model with optimal parameters
gbm = lgb.LGBMRegressor(objective="regression", boosting_type='gbdt', random_state=2020, 
                        n_jobs=-1, max_depth=10, n_estimators=300, learning_rate=0.05,
                        num_leaves= 35, reg_alpha = 0.5, reg_lambda=0.5)
gbm.fit(training_x,training_y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=10,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=35,
              objective='regression', random_state=2020, reg_alpha=0.5,
              reg_lambda=0.5, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [44]:
predicted_ratings = gbm.predict(testing_x)
predicted_ratings

array([3.93292405, 3.4674965 , 3.73967371, ..., 2.01322249, 3.27536353,
       3.99384755])

In [46]:
rmse = sqrt(mean_squared_error(predicted_ratings, testing_y))
rmse

1.3060903296639528

In [47]:
feature_importances = dict()
for i in range(20):
    feature_importances[i] = gbm.feature_importances_[i]

In [48]:
sorted(feature_importances.items(), key=lambda x:x[1], reverse=True)

[(1, 1416),
 (0, 1029),
 (19, 927),
 (11, 772),
 (16, 732),
 (17, 706),
 (12, 647),
 (2, 551),
 (10, 549),
 (15, 541),
 (5, 524),
 (6, 485),
 (4, 479),
 (9, 431),
 (14, 211),
 (7, 200),
 (3, 0),
 (8, 0),
 (13, 0),
 (18, 0)]